# Importando bases

In [14]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier

In [15]:
df_bolsonaro_train = pd.read_excel('Bolsonaro_treino_sem_stop2.xlsx')  
df_lula_train = pd.read_excel('Lula_treino_sem_stop2.xlsx') 
df_simone_train = pd.read_excel('Simone_treino_sem_stop2.xlsx') 
df_ciro_train = pd.read_excel('Ciro_treino_sem_stop2.xlsx') 

In [16]:
df_bolsonaro = pd.read_excel('Bolsonaro_todos_sem_stop.xlsx', index_col=0)
df_lula = pd.read_excel('Lula_todos_sem_stop.xlsx', index_col=0)
df_simone = pd.read_excel('Simone_todos_sem_stop.xlsx', index_col=0)
df_ciro = pd.read_excel('Ciro_todos_sem_stop.xlsx', index_col=0)



In [17]:
df_lula_train = df_lula_train[df_lula_train['Sentimento']!=' ']

In [18]:
print(df_lula_train['Sentimento'].value_counts())
print(df_bolsonaro_train['Sentimento'].value_counts())
print(df_simone_train['Sentimento'].value_counts())
print(df_ciro_train['Sentimento'].value_counts())

p    10335
n     1773
x       81
Name: Sentimento, dtype: int64
p    5065
n     848
x     108
Name: Sentimento, dtype: int64
p    120
x    109
n     50
Name: Sentimento, dtype: int64
p    6807
x     105
n      81
Name: Sentimento, dtype: int64


In [19]:
def train_test_under(df):
    tamanho = df['Sentimento'].value_counts()[2]-5
    
    df_train = df[df['Sentimento']=='p'].sample(n = tamanho)
    df_train = df_train.append(df[df['Sentimento']=='n'].sample(n = tamanho))
    df_train = df_train.append(df[df['Sentimento']=='x'].sample(n = tamanho))
    
    df_teste = df.drop(df_train.index)
    
    X_bw_train, bw = Bag_of_words(df_train)
    X_tfidf_train, tfidf = tfidf(df_train)
    Y_train = df_train['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int)
    
    X_bw_teste = Bag_of_words_teste(df_teste, bw )
    X_tfidf_teste = tfidf_teste(df_teste, tfidf)
    Y_teste = df_teste['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int)
    
    return X_bw_train, X_tfidf_train, Y_train, X_bw_teste, X_tfidf_teste, Y_teste, bw, tfidf, df_train, df_teste


In [20]:
# Bag Of Words

def Bag_of_words(df):
    matrix = CountVectorizer()
    text = []
    for i in df.index:
        text.append("".join(df['novo_texto'][i]))
    X = matrix.fit_transform(text)
    return X, matrix

def Bag_of_words_teste(df, matrix):
    text = []
    for i in df.index:
        text.append("".join(df['novo_texto'][i]))
    X = matrix.transform(text)
    return X

def tfidf_(df):
    matrix = TfidfVectorizer()
    text = []
    for i in df.index:
        text.append("".join(df['novo_texto'][i]))
    X = matrix.fit_transform(text)
    return X, matrix

def tfidf_teste(df, matrix):
    text = []
    for i in df.index:
        text.append("".join(df['novo_texto'][i]))
    X = matrix.transform(text)
    return X

def train_test_under_bw(df):
    tamanho = df['Sentimento'].value_counts()[2]-2
    
    df_train = df[df['Sentimento']=='p'].sample(n = tamanho)
    df_train = df_train.append(df[df['Sentimento']=='n'].sample(n = tamanho))
    df_train = df_train.append(df[df['Sentimento']=='x'].sample(n = tamanho))
    
    df_teste = df.drop(df_train.index)
    
    X_bw_train, bw = Bag_of_words(df_train)
    Y_train = df_train['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int)
    
    X_bw_teste = Bag_of_words_teste(df_teste, bw )
    
    Y_teste = df_teste['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int)
    
    return X_bw_train, Y_train, X_bw_teste, Y_teste, bw, df_train, df_teste

def train_test_under_tfidf(df):
    tamanho = df['Sentimento'].value_counts()[2]-2
    
    df_train = df[df['Sentimento']=='p'].sample(n = tamanho)
    df_train = df_train.append(df[df['Sentimento']=='n'].sample(n = tamanho))
    df_train = df_train.append(df[df['Sentimento']=='x'].sample(n = tamanho))
    
    df_teste = df.drop(df_train.index)
    
    X_tfidf_train, tfidf = tfidf_(df_train)
    Y_train = df_train['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int)
    
    X_tfidf_teste = tfidf_teste(df_teste, tfidf)
    Y_teste = df_teste['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int)
    
    return X_tfidf_train, Y_train, X_tfidf_teste, Y_teste, tfidf, df_train, df_teste

def train_model_bw(model, df_train_tudo, num_tests):
    X_train, Y_train, X_teste, Y_teste, bw, df_train, df_teste = train_test_under_bw(df_train_tudo)
    
    model2 = model.fit(X_train.toarray(), Y_train)
    predict = model2.predict(X_teste.toarray())
    accuracy = accuracy_score(Y_teste, predict)
    precision = precision_score(Y_teste, predict, average='macro')
    recall = recall_score(Y_teste, predict, average='macro')
    f1 = f1_score(Y_teste, predict, average='macro')
    score = accuracy + precision + recall + f1
    
    matrix_confusion = confusion_matrix(Y_teste, predict,labels=[-1, 0, 1])
    
    errados = matrix_confusion[0][2]+ matrix_confusion[2][0]
    
    model2_fim, bw_fim, df_train_fim, df_teste_fim, score_fim, errados_fim = model2, bw, df_train, df_teste, score, errados
    for i in range(0,num_tests):
        X_train, Y_train, X_teste, Y_teste, bw, df_train, df_teste = train_test_under_bw(df_train_tudo)

        model2 = model.fit(X_train.toarray(), Y_train)
        predict = model2.predict(X_teste.toarray())
        accuracy = accuracy_score(Y_teste, predict)
        precision = precision_score(Y_teste, predict, average='macro')
        recall = recall_score(Y_teste, predict, average='macro')
        f1 = f1_score(Y_teste, predict, average='macro')
        score = accuracy + precision + recall + f1

        matrix_confusion = confusion_matrix(Y_teste, predict,labels=[-1, 0, 1])
        accuracy_fim, precision_fim,recall_fim, f1_fim, matrix_confusion_fim = accuracy, precision,recall,f1,matrix_confusion 
        errados = matrix_confusion[0][2]+ matrix_confusion[2][0]
        
        if((score_fim<score) & (errados_fim>errados)):
            accuracy_fim, precision_fim,recall_fim, f1_fim, matrix_confusion_fim = accuracy, precision,recall,f1,matrix_confusion 
            model2_fim, bw_fim, df_train_fim, df_teste_fim, score_fim, errados_fim = model2, bw, df_train, df_teste, score, errados
    
    print('accuracy:')
    print(accuracy_fim)
    print('precision:')
    print(precision_fim)
    print('recall:')
    print(recall_fim)
    print('f1:')
    print(f1_fim)
    print('matrix_confusion:')
    print(matrix_confusion_fim)
    print(score_fim/4)
    print(errados_fim)
    return model2_fim, bw_fim, df_train_fim, df_teste_fim     
    
def train_model_tfidf(model, df_train_tudo, num_tests):
    X_train, Y_train, X_teste, Y_teste, tfidf, df_train, df_teste = train_test_under_tfidf(df_train_tudo)
    
    model2 = model.fit(X_train.toarray(), Y_train)
    predict = model2.predict(X_teste.toarray())
    accuracy = accuracy_score(Y_teste, predict)
    precision = precision_score(Y_teste, predict, average='macro')
    recall = recall_score(Y_teste, predict, average='macro')
    f1 = f1_score(Y_teste, predict, average='macro')
    score = accuracy + precision + recall + f1
    
    matrix_confusion = confusion_matrix(Y_teste, predict,labels=[-1, 0, 1])
    accuracy_fim, precision_fim,recall_fim, f1_fim, matrix_confusion_fim = accuracy, precision,recall,f1,matrix_confusion 
    errados = matrix_confusion[0][2]+ matrix_confusion[2][0]
    
    model2_fim, tfidf_fim, df_train_fim, df_teste_fim, score_fim, errados_fim = model2, tfidf, df_train, df_teste, score, errados
    for i in range(0,num_tests):
        X_train, Y_train, X_teste, Y_teste, tfidf, df_train, df_teste = train_test_under_tfidf(df_train_tudo)

        model2 = model.fit(X_train.toarray(), Y_train)
        predict = model2.predict(X_teste.toarray())
        accuracy = accuracy_score(Y_teste, predict)
        precision = precision_score(Y_teste, predict, average='macro')
        recall = recall_score(Y_teste, predict, average='macro')
        f1 = f1_score(Y_teste, predict, average='macro')
        score = accuracy + precision + recall + f1

        matrix_confusion = confusion_matrix(Y_teste, predict,labels=[-1, 0, 1])

        errados = matrix_confusion[0][2]+ matrix_confusion[2][0]
        
        if((score_fim<score) & (errados_fim>errados)):
            accuracy_fim, precision_fim,recall_fim, f1_fim, matrix_confusion_fim = accuracy, precision,recall,f1,matrix_confusion 
            model2_fim, tfidf_fim, df_train_fim, df_teste_fim, score_fim, errados_fim = model2, tfidf, df_train, df_teste, score, errados
    
    print('accuracy:')
    print(accuracy_fim)
    print('precision:')
    print(precision_fim)
    print('recall:')
    print(recall_fim)
    print('f1:')
    print(f1_fim)
    print('matrix_confusion:')
    print(matrix_confusion_fim)
    print(score_fim/4)
    print(errados_fim)
    return model2_fim, tfidf_fim, df_train_fim, df_teste_fim  

def AnaliseSentimentos_bw(df, bw,model, name_colum):
    i1=0
    for i in range(5000,len(df),5000):
        if (i==5000):
            predict = model.predict(Bag_of_words_teste(df[i1:i], bw).toarray())
        else:
            predict = np.append(predict, model.predict(Bag_of_words_teste(df[i1:i], bw).toarray()))
        i1=i
        
    predict = np.append(predict,model.predict(Bag_of_words_teste(df[i1:], bw).toarray()))

    df[name_colum] = predict

    print(np.unique(predict, return_counts=True))
    return df

def AnaliseSentimentos_tfidf(df, tfidf,model, name_colum):
    i1=0
    for i in range(5000,len(df),5000):
        if (i==5000):
            predict = model.predict(tfidf_teste(df[i1:i], tfidf).toarray())
        else:
            predict = np.append(predict, model.predict(tfidf_teste(df[i1:i], tfidf).toarray()))
        i1=i
        
    predict = np.append(predict,model.predict(tfidf_teste(df[i1:], tfidf).toarray()))

    df[name_colum] = predict

    print(np.unique(predict, return_counts=True))
    return df

## Bolsonaro

### Bolsonaro - Arvore de Decisao - bw

In [21]:
%%time
model_tree_bolsonaro_bw, tree_bolsonaro_bw, df_train_tree_bolsonaro_bw, df_teste_tree_bolsonaro_bw = \
train_model_bw(tree.DecisionTreeClassifier(), df_bolsonaro_train, 100)
model_tree_bolsonaro_bw = model_tree_bolsonaro_bw.fit(Bag_of_words_teste(df_train_tree_bolsonaro_bw, tree_bolsonaro_bw), df_train_tree_bolsonaro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_bw(df_bolsonaro, tree_bolsonaro_bw,model_tree_bolsonaro_bw, 'predict_bw_ArvoreDeDecisao')


accuracy:
0.503594599333684
precision:
0.3767103884854037
recall:
0.5158946669065493
f1:
0.31612868357547075
matrix_confusion:
[[ 409  125  208]
 [   1    1    0]
 [1565  932 2462]]
0.5042903153856906
1010
(array([-1,  0,  1]), array([140918, 215693, 589827], dtype=int64))
Wall time: 1min 9s


### Bolsonaro - Arvore de Decisao - tfidf

In [22]:
%%time
model_tree_bolsonaro_tfidf, tree_bolsonaro_tfidf, df_train_tree_bolsonaro_tfidf, df_teste_tree_bolsonaro_tfidf = \
train_model_tfidf(tree.DecisionTreeClassifier(), df_bolsonaro_train, 100)
model_tree_bolsonaro_tfidf = model_tree_bolsonaro_tfidf.fit(tfidf_teste(df_train_tree_bolsonaro_tfidf, tree_bolsonaro_tfidf), df_train_tree_bolsonaro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_tfidf(df_bolsonaro, tree_bolsonaro_tfidf,model_tree_bolsonaro_tfidf, 'predict_tfidf_ArvoreDeDecisao')


accuracy:
0.6287918639312643
precision:
0.38600979257686746
recall:
0.4970349860772078
f1:
0.35212453741456384
matrix_confusion:
[[ 234  148  360]
 [   0    1    1]
 [ 686  922 3351]]
0.46599029499997585
1046
(array([-1,  0,  1]), array([121064, 248564, 576810], dtype=int64))
Wall time: 1min 8s


### Bolsonaro - Naive Bayes - bw

In [23]:
%%time
model_nb_bolsonaro_bw, nb_bolsonaro_bw, df_train_nb_bolsonaro_bw, df_teste_nb_bolsonaro_bw = \
train_model_bw(GaussianNB(), df_bolsonaro_train, 100)
model_nb_bolsonaro_bw = model_nb_bolsonaro_bw.fit(Bag_of_words_teste(df_train_nb_bolsonaro_bw, nb_bolsonaro_bw).toarray(), df_train_nb_bolsonaro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_bw(df_bolsonaro, nb_bolsonaro_bw,model_nb_bolsonaro_bw, 'predict_bw_NaiveBayes')


accuracy:
0.49132035770647026
precision:
0.3699379377435046
recall:
0.6491374826134954
f1:
0.30708618277530647
matrix_confusion:
[[ 334  165  243]
 [   0    2    0]
 [1353 1140 2466]]
0.43047422402768193
1213
(array([-1,  0,  1]), array([202913, 404396, 339129], dtype=int64))
Wall time: 2min 27s


### Bolsonaro - Naive Bayes - tfidf

In [24]:
%%time
model_nb_bolsonaro_tfidf, nb_bolsonaro_tfidf, df_train_nb_bolsonaro_tfidf, df_teste_nb_bolsonaro_tfidf = \
train_model_tfidf(GaussianNB(), df_bolsonaro_train, 100)
model_nb_bolsonaro_tfidf = model_nb_bolsonaro_tfidf.fit(tfidf_teste(df_train_nb_bolsonaro_tfidf, nb_bolsonaro_tfidf).toarray(), df_train_nb_bolsonaro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_tfidf(df_bolsonaro, nb_bolsonaro_tfidf,model_nb_bolsonaro_tfidf, 'predict_tfidf_NaiveBayes')


accuracy:
0.476766614062774
precision:
0.3722568641463389
recall:
0.4628242695222115
f1:
0.30306571090469653
matrix_confusion:
[[ 297  205  240]
 [   0    1    1]
 [1140 1398 2421]]
0.40372836465900525
1380
(array([-1,  0,  1]), array([229336, 323876, 393226], dtype=int64))
Wall time: 2min 32s


### Bolsonaro - Random Forest - bw

In [25]:
%%time
model_RandomForest_bolsonaro_bw, RandomForest_bolsonaro_bw, df_train_RandomForest_bolsonaro_bw, df_teste_RandomForest_bolsonaro_bw = \
train_model_bw(RandomForestClassifier(), df_bolsonaro_train, 100)
model_RandomForest_bolsonaro_bw = model_RandomForest_bolsonaro_bw.fit(Bag_of_words_teste(df_train_RandomForest_bolsonaro_bw, RandomForest_bolsonaro_bw).toarray(), df_train_RandomForest_bolsonaro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_bw(df_bolsonaro, RandomForest_bolsonaro_bw,model_RandomForest_bolsonaro_bw, 'predict_bw_RandomForest')


accuracy:
0.6344029458179905
precision:
0.4058181635506945
recall:
0.6501226499343131
f1:
0.35651308446450747
matrix_confusion:
[[ 193  186  363]
 [   0    2    0]
 [ 426 1110 3423]]
0.5178657284248911
678
(array([-1,  0,  1]), array([ 66473, 293132, 586833], dtype=int64))
Wall time: 3min 12s


### Bolsonaro - Random Forest - tfidf

In [26]:
%%time
model_RandomForest_bolsonaro_tfidf, RandomForest_bolsonaro_tfidf, df_train_RandomForest_bolsonaro_tfidf, df_teste_RandomForest_bolsonaro_tfidf = \
train_model_tfidf(RandomForestClassifier(), df_bolsonaro_train, 100)
model_RandomForest_bolsonaro_tfidf = model_RandomForest_bolsonaro_tfidf.fit(tfidf_teste(df_train_RandomForest_bolsonaro_tfidf, RandomForest_bolsonaro_tfidf).toarray(), df_train_RandomForest_bolsonaro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_tfidf(df_bolsonaro, RandomForest_bolsonaro_tfidf,model_RandomForest_bolsonaro_tfidf, 'predict_tfidf_RandomForest')


accuracy:
0.4665965281430826
precision:
0.4181166133218001
recall:
0.62094086151546
f1:
0.3287850794186349
matrix_confusion:
[[ 285  268  189]
 [   0    2    0]
 [ 586 1999 2374]]
0.45860977059974445
775
(array([-1,  0,  1]), array([155915, 365016, 425507], dtype=int64))
Wall time: 3min


### Bolsonaro - SVM - bw

In [27]:
%%time
model_svc_bolsonaro_bw, svc_bolsonaro_bw, df_train_svc_bolsonaro_bw, df_teste_svc_bolsonaro_bw = \
train_model_bw(LinearSVC(), df_bolsonaro_train, 100)
model_svc_bolsonaro_bw = model_svc_bolsonaro_bw.fit(Bag_of_words_teste(df_train_svc_bolsonaro_bw, svc_bolsonaro_bw).toarray(), df_train_svc_bolsonaro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_bw(df_bolsonaro, svc_bolsonaro_bw,model_svc_bolsonaro_bw, 'predict_bw_SVM')

accuracy:
0.4853585832018236
precision:
0.37546213186886074
recall:
0.6415038173761592
f1:
0.3083932140770262
matrix_confusion:
[[ 320  199  223]
 [   0    2    0]
 [1214 1299 2446]]
0.43441421221006987
1090
(array([-1,  0,  1]), array([197530, 318218, 430690], dtype=int64))
Wall time: 1min 11s


### Bolsonaro - SVM - tfidf

In [28]:
%%time
model_svc_bolsonaro_tfidf, svc_bolsonaro_tfidf, df_train_svc_bolsonaro_tfidf, df_teste_svc_bolsonaro_tfidf = \
train_model_tfidf(LinearSVC(), df_bolsonaro_train, 100)
model_svc_bolsonaro_tfidf = model_svc_bolsonaro_tfidf.fit(tfidf_teste(df_train_svc_bolsonaro_tfidf, svc_bolsonaro_tfidf).toarray(), df_train_svc_bolsonaro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_tfidf(df_bolsonaro, svc_bolsonaro_tfidf,model_svc_bolsonaro_tfidf, 'predict_tfidf_SVM')

accuracy:
0.5430475188497282
precision:
0.3907150918117806
recall:
0.5042773926792692
f1:
0.3387351810824742
matrix_confusion:
[[ 339  180  223]
 [   0    1    1]
 [1036 1166 2757]]
0.44419379610581305
1259
(array([-1,  0,  1]), array([247462, 298824, 400152], dtype=int64))
Wall time: 1min 6s


### Bolsonaro - MLP - bw

In [29]:
%%time
model_mlp_bolsonaro_bw, mlp_bolsonaro_bw, df_train_mlp_bolsonaro_bw, df_teste_mlp_bolsonaro_bw = \
train_model_bw(MLPClassifier(max_iter=400), df_bolsonaro_train, 100)
model_mlp_bolsonaro_bw = model_mlp_bolsonaro_bw.fit(Bag_of_words_teste(df_train_mlp_bolsonaro_bw, mlp_bolsonaro_bw).toarray(), df_train_mlp_bolsonaro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_bw(df_bolsonaro, mlp_bolsonaro_bw,model_mlp_bolsonaro_bw, 'predict_bw_MLP')

accuracy:
0.4844818516570226
precision:
0.37679133324331754
recall:
0.6358194698776146
f1:
0.3099916891060748
matrix_confusion:
[[ 306  193  243]
 [   0    2    0]
 [1090 1414 2455]]
0.4386815243563395
1254
(array([-1,  0,  1]), array([222892, 277241, 446305], dtype=int64))
Wall time: 9min 29s


### Bolsonaro - MLP - tfidf

In [30]:
%%time
model_mlp_bolsonaro_tfidf, mlp_bolsonaro_tfidf, df_train_mlp_bolsonaro_tfidf, df_teste_mlp_bolsonaro_tfidf = \
train_model_tfidf(MLPClassifier(max_iter=400), df_bolsonaro_train, 100)
model_mlp_bolsonaro_tfidf = model_mlp_bolsonaro_tfidf.fit(Bag_of_words_teste(df_train_mlp_bolsonaro_tfidf, mlp_bolsonaro_tfidf).toarray(), df_train_mlp_bolsonaro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_tfidf(df_bolsonaro, mlp_bolsonaro_tfidf,model_mlp_bolsonaro_tfidf, 'predict_tfidf_MLP')

accuracy:
0.49728213221111695
precision:
0.38044582242629493
recall:
0.661355369193605
f1:
0.31756110491865946
matrix_confusion:
[[ 360  166  216]
 [   0    2    0]
 [1275 1210 2474]]
0.4641611071874191
1491
(array([-1,  0,  1]), array([293643, 270755, 382040], dtype=int64))
Wall time: 10min 19s


### Bolsonaro - KNN - bw

In [31]:
%%time
model_knn_bolsonaro_bw, knn_bolsonaro_bw, df_train_knn_bolsonaro_bw, df_teste_knn_bolsonaro_bw = \
train_model_bw(KNeighborsClassifier(), df_bolsonaro_train, 100)
model_knn_bolsonaro_bw = model_knn_bolsonaro_bw.fit(Bag_of_words_teste(df_train_knn_bolsonaro_bw, knn_bolsonaro_bw).toarray(), df_train_knn_bolsonaro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_bw(df_bolsonaro, knn_bolsonaro_bw,model_knn_bolsonaro_bw, 'predict_bw_KNN')

accuracy:
0.21322111169559882
precision:
0.34825107274945166
recall:
0.49512027375603035
f1:
0.1488040232507016
matrix_confusion:
[[ 646   36   60]
 [   1    1    0]
 [4052  338  569]]
0.526992420593109
934
(array([-1,  0,  1]), array([ 62561,  48210, 835667], dtype=int64))
Wall time: 2min 33s


### Bolsonaro - KNN - tfidf

In [32]:
%%time
model_knn_bolsonaro_tfidf, knn_bolsonaro_tfidf, df_train_knn_bolsonaro_tfidf, df_teste_knn_bolsonaro_tfidf = \
train_model_tfidf(KNeighborsClassifier(), df_bolsonaro_train, 100)
model_knn_bolsonaro_tfidf = model_knn_bolsonaro_tfidf.fit(Bag_of_words_teste(df_train_knn_bolsonaro_tfidf, knn_bolsonaro_tfidf).toarray(), df_train_knn_bolsonaro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_tfidf(df_bolsonaro, knn_bolsonaro_tfidf,model_knn_bolsonaro_tfidf, 'predict_tfidf_KNN')

accuracy:
0.4273189549359986
precision:
0.3610571509009653
recall:
0.6234569109102547
f1:
0.27800260801095866
matrix_confusion:
[[ 331  189  222]
 [   0    2    0]
 [1536 1319 2104]]
0.4224589061895443
1758
(array([-1,  0,  1]), array([365699, 289089, 291650], dtype=int64))
Wall time: 3min 5s


# Lula

### Lula - Arvore de Decisao - bw

In [33]:
%%time
model_tree_lula_bw, tree_lula_bw, df_train_tree_lula_bw, df_teste_tree_lula_bw = \
train_model_bw(tree.DecisionTreeClassifier(), df_lula_train, 100)
model_tree_lula_bw = model_tree_lula_bw.fit(Bag_of_words_teste(df_train_tree_lula_bw, tree_lula_bw), df_train_tree_lula_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_bw(df_lula, tree_lula_bw,model_tree_lula_bw, 'predict_bw_ArvoreDeDecisao')


accuracy:
0.4764892904953146
precision:
0.41465563044214376
recall:
0.6100268007178374
f1:
0.3247910264933757
matrix_confusion:
[[ 558  721  415]
 [   0    2    0]
 [1195 3926 5135]]
0.46704406845169755
1679
(array([-1,  0,  1]), array([180896, 444691, 306143], dtype=int64))
Wall time: 1min 19s


### Lula - Arvore de Decisao - tfidf

In [34]:
%%time
model_tree_lula_tfidf, tree_lula_tfidf, df_train_tree_lula_tfidf, df_teste_tree_lula_tfidf = \
train_model_tfidf(tree.DecisionTreeClassifier(), df_lula_train, 100)
model_tree_lula_tfidf = model_tree_lula_tfidf.fit(tfidf_teste(df_train_tree_lula_tfidf, tree_lula_tfidf), df_train_tree_lula_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_tfidf(df_lula, tree_lula_tfidf,model_tree_lula_tfidf, 'predict_tfidf_ArvoreDeDecisao')


accuracy:
0.5148929049531459
precision:
0.40036197644608373
recall:
0.6295445029135286
f1:
0.33177094714566574
matrix_confusion:
[[ 586  589  519]
 [   0    2    0]
 [1464 3226 5566]]
0.46914258286460603
1983
(array([-1,  0,  1]), array([177805, 360098, 393827], dtype=int64))
Wall time: 1min 20s


### Lula - Naive Bayes - bw

In [35]:
%%time
model_nb_lula_bw, nb_lula_bw, df_train_nb_lula_bw, df_teste_nb_lula_bw = \
train_model_bw(GaussianNB(), df_lula_train, 100)
model_nb_lula_bw = model_nb_lula_bw.fit(Bag_of_words_teste(df_train_nb_lula_bw, nb_lula_bw).toarray(), df_train_nb_lula_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_bw(df_lula, nb_lula_bw,model_nb_lula_bw, 'predict_bw_NaiveBayes')


accuracy:
0.35081994645247655
precision:
0.36878210106120163
recall:
0.5743515779592223
f1:
0.2554935872805243
matrix_confusion:
[[ 638  667  389]
 [   0    2    0]
 [2480 4223 3553]]
0.43451889798955995
2293
(array([-1,  0,  1]), array([222587, 410034, 299109], dtype=int64))
Wall time: 2min 48s


### Lula - Naive Bayes - tfidf

In [36]:
%%time
model_nb_lula_tfidf, nb_lula_tfidf, df_train_nb_lula_tfidf, df_teste_nb_lula_tfidf = \
train_model_tfidf(GaussianNB(), df_lula_train, 100)
model_nb_lula_tfidf = model_nb_lula_tfidf.fit(tfidf_teste(df_train_nb_lula_tfidf, nb_lula_tfidf).toarray(), df_train_nb_lula_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_tfidf(df_lula, nb_lula_tfidf,model_nb_lula_tfidf, 'predict_tfidf_NaiveBayes')


accuracy:
0.47849732262382866
precision:
0.37279281697620736
recall:
0.6168848820068505
f1:
0.30383120666664915
matrix_confusion:
[[ 595  541  558]
 [   0    2    0]
 [2159 2975 5122]]
0.4430015570683839
2717
(array([-1,  0,  1]), array([244317, 342539, 344874], dtype=int64))
Wall time: 2min 53s


### Lula - Random Forest - bw

In [37]:
%%time
model_RandomForest_lula_bw, RandomForest_lula_bw, df_train_RandomForest_lula_bw, df_teste_RandomForest_lula_bw = \
train_model_bw(RandomForestClassifier(), df_lula_train, 100)
model_RandomForest_lula_bw = model_RandomForest_lula_bw.fit(Bag_of_words_teste(df_train_RandomForest_lula_bw, RandomForest_lula_bw).toarray(), df_train_RandomForest_lula_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_bw(df_lula, RandomForest_lula_bw,model_RandomForest_lula_bw, 'predict_bw_RandomForest')


accuracy:
0.47740963855421686
precision:
0.4059287166528174
recall:
0.43340103734019486
f1:
0.31611304013282127
matrix_confusion:
[[ 495  785  414]
 [   0    1    1]
 [1204 3842 5210]]
0.44954464556597384
1001
(array([-1,  0,  1]), array([ 42383, 609923, 279424], dtype=int64))
Wall time: 3min 23s


### Lula - Random Forest - tfidf

In [38]:
%%time
model_RandomForest_lula_tfidf, RandomForest_lula_tfidf, df_train_RandomForest_lula_tfidf, df_teste_RandomForest_lula_tfidf = \
train_model_tfidf(RandomForestClassifier(), df_lula_train, 100)
model_RandomForest_lula_tfidf = model_RandomForest_lula_tfidf.fit(tfidf_teste(df_train_RandomForest_lula_tfidf, RandomForest_lula_tfidf).toarray(), df_train_RandomForest_lula_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_tfidf(df_lula, RandomForest_lula_tfidf,model_RandomForest_lula_tfidf, 'predict_tfidf_RandomForest')


accuracy:
0.5150602409638554
precision:
0.42129696951422146
recall:
0.6245170851698295
f1:
0.3396559326558881
matrix_confusion:
[[ 555  731  408]
 [   0    2    0]
 [1120 3537 5599]]
0.4751325570759486
1528
(array([-1,  0,  1]), array([178809, 461001, 291920], dtype=int64))
Wall time: 3min 16s


### lula - SVM - bw

In [39]:
%%time
model_svc_lula_bw, svc_lula_bw, df_train_svc_lula_bw, df_teste_svc_lula_bw = \
train_model_bw(LinearSVC(), df_lula_train, 100)
model_svc_lula_bw = model_svc_lula_bw.fit(Bag_of_words_teste(df_train_svc_lula_bw, svc_lula_bw).toarray(), df_train_svc_lula_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_bw(df_lula, svc_lula_bw,model_svc_lula_bw, 'predict_bw_SVM')

accuracy:
0.47448125836680055
precision:
0.3918018974701742
recall:
0.6281380062758591
f1:
0.31504002277927035
matrix_confusion:
[[ 673  626  395]
 [   0    2    0]
 [2039 3221 4996]]
0.48022278195083445
1702
(array([-1,  0,  1]), array([113347, 449744, 368639], dtype=int64))
Wall time: 1min 27s


### lula - SVM - tfidf

In [40]:
%%time
model_svc_lula_tfidf, svc_lula_tfidf, df_train_svc_lula_tfidf, df_teste_svc_lula_tfidf = \
train_model_tfidf(LinearSVC(), df_lula_train, 100)
model_svc_lula_tfidf = model_svc_lula_tfidf.fit(tfidf_teste(df_train_svc_lula_tfidf, svc_lula_tfidf).toarray(), df_train_svc_lula_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_tfidf(df_lula, svc_lula_tfidf,model_svc_lula_tfidf, 'predict_tfidf_SVM')

accuracy:
0.5686077643908969
precision:
0.4069105673152465
recall:
0.6745582662739804
f1:
0.3584647700079935
matrix_confusion:
[[ 733  434  527]
 [   0    2    0]
 [1710 2485 6061]]
0.5021353419970294
2237
(array([-1,  0,  1]), array([230790, 335056, 365884], dtype=int64))
Wall time: 1min 24s


### Lula - MLP - bw

In [41]:
%%time
model_mlp_lula_bw, mlp_lula_bw, df_train_mlp_lula_bw, df_teste_mlp_lula_bw = \
train_model_bw(MLPClassifier(max_iter=400), df_lula_train, 100)
model_mlp_lula_bw = model_mlp_lula_bw.fit(Bag_of_words_teste(df_train_mlp_lula_bw, mlp_lula_bw).toarray(), df_train_mlp_lula_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_bw(df_lula, mlp_lula_bw,model_mlp_lula_bw, 'predict_bw_MLP')

accuracy:
0.47832998661311915
precision:
0.38110468377318923
recall:
0.6429390676217367
f1:
0.31165497153880767
matrix_confusion:
[[ 754  483  457]
 [   0    2    0]
 [2567 2728 4961]]
0.4806175903860879
2262
(array([-1,  0,  1]), array([203289, 378076, 350365], dtype=int64))
Wall time: 8min 17s


### Lula - MLP - tfidf

In [42]:
%%time
model_mlp_lula_tfidf, mlp_lula_tfidf, df_train_mlp_lula_tfidf, df_teste_mlp_lula_tfidf = \
train_model_tfidf(MLPClassifier(max_iter=400), df_lula_train, 100)
model_mlp_lula_tfidf = model_mlp_lula_tfidf.fit(Bag_of_words_teste(df_train_mlp_lula_tfidf, mlp_lula_tfidf).toarray(), df_train_mlp_lula_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_tfidf(df_lula, mlp_lula_tfidf,model_mlp_lula_tfidf, 'predict_tfidf_MLP')

accuracy:
0.46971218206157966
precision:
0.3884990310417056
recall:
0.6280924200368251
f1:
0.31219269455393833
matrix_confusion:
[[ 684  589  421]
 [   0    2    0]
 [2123 3205 4928]]
0.4496240819235122
2544
(array([-1,  0,  1]), array([281232, 373494, 277004], dtype=int64))
Wall time: 9min 8s


### Lula - KNN - bw

In [43]:
%%time
model_knn_lula_bw, knn_lula_bw, df_train_knn_lula_bw, df_teste_knn_lula_bw = \
train_model_bw(KNeighborsClassifier(), df_lula_train, 100)
model_knn_lula_bw = model_knn_lula_bw.fit(Bag_of_words_teste(df_train_knn_lula_bw, knn_lula_bw).toarray(), df_train_knn_lula_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_bw(df_lula, knn_lula_bw,model_knn_lula_bw, 'predict_bw_KNN')

accuracy:
0.15311244979919678
precision:
0.36309428492027557
recall:
0.2845922426035176
f1:
0.13969493060000834
matrix_confusion:
[[ 356 1257   81]
 [   0    1    1]
 [2153 6630 1473]]
0.37473675023031083
551
(array([-1,  0,  1]), array([ 17448, 764571, 149711], dtype=int64))
Wall time: 3min 12s


### Lula - KNN - tfidf

In [44]:
%%time
model_knn_lula_tfidf, knn_lula_tfidf, df_train_knn_lula_tfidf, df_teste_knn_lula_tfidf = \
train_model_tfidf(KNeighborsClassifier(), df_lula_train, 100)
model_knn_lula_tfidf = model_knn_lula_tfidf.fit(Bag_of_words_teste(df_train_knn_lula_tfidf, knn_lula_tfidf).toarray(), df_train_knn_lula_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_tfidf(df_lula, knn_lula_tfidf,model_knn_lula_tfidf, 'predict_tfidf_KNN')

accuracy:
0.5387382864792504
precision:
0.3895578010075373
recall:
0.5096271383322866
f1:
0.3388240957521333
matrix_confusion:
[[ 814  378  502]
 [   1    1    0]
 [2438 2194 5624]]
0.4441868303928019
2940
(array([-1,  0,  1]), array([287818, 295979, 347933], dtype=int64))
Wall time: 3min 31s


# Simone

### Simone - Arvore de Decisao - bw

In [45]:
%%time
model_tree_simone_bw, tree_simone_bw, df_train_tree_simone_bw, df_teste_tree_simone_bw = \
train_model_bw(tree.DecisionTreeClassifier(), df_simone_train, 100)
model_tree_simone_bw = model_tree_simone_bw.fit(Bag_of_words_teste(df_train_tree_simone_bw, tree_simone_bw), df_train_tree_simone_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_bw(df_simone, tree_simone_bw,model_tree_simone_bw, 'predict_bw_ArvoreDeDecisao')


accuracy:
0.48148148148148145
precision:
0.4775311025311025
recall:
0.49218275652701876
f1:
0.39195174516275433
matrix_confusion:
[[ 1  1  0]
 [18 35  8]
 [23 20 29]]
0.5824212407655569
8
(array([-1,  0,  1]), array([33816, 44920, 52426], dtype=int64))
Wall time: 6.6 s


### Simone - Arvore de Decisao - tfidf

In [46]:
%%time
model_tree_simone_tfidf, tree_simone_tfidf, df_train_tree_simone_tfidf, df_teste_tree_simone_tfidf = \
train_model_tfidf(tree.DecisionTreeClassifier(), df_simone_train, 100)
model_tree_simone_tfidf = model_tree_simone_tfidf.fit(tfidf_teste(df_train_tree_simone_tfidf, tree_simone_tfidf), df_train_tree_simone_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_tfidf(df_simone, tree_simone_tfidf,model_tree_simone_tfidf, 'predict_tfidf_ArvoreDeDecisao')


accuracy:
0.6666666666666666
precision:
0.5335485133020345
recall:
0.7699605343047966
f1:
0.535227599447783
matrix_confusion:
[[ 2  0  0]
 [ 8 35 18]
 [ 6 13 53]]
0.6263508284303202
6
(array([-1,  0,  1]), array([36998, 53451, 40713], dtype=int64))
Wall time: 6.78 s


### Simone - Naive Bayes - bw

In [47]:
%%time
model_nb_simone_bw, nb_simone_bw, df_train_nb_simone_bw, df_teste_nb_simone_bw = \
train_model_bw(GaussianNB(), df_simone_train, 100)
model_nb_simone_bw = model_nb_simone_bw.fit(Bag_of_words_teste(df_train_nb_simone_bw, nb_simone_bw).toarray(), df_train_nb_simone_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_bw(df_simone, nb_simone_bw,model_nb_simone_bw, 'predict_bw_NaiveBayes')


accuracy:
0.6
precision:
0.5140238183716445
recall:
0.7349726775956285
f1:
0.48219167382644695
matrix_confusion:
[[ 2  0  0]
 [ 8 43 10]
 [16 20 36]]
0.5863891294086958
10
(array([-1,  0,  1]), array([29535, 61064, 40563], dtype=int64))
Wall time: 9.84 s


### Simone - Naive Bayes - tfidf

In [48]:
%%time
model_nb_simone_tfidf, nb_simone_tfidf, df_train_nb_simone_tfidf, df_teste_nb_simone_tfidf = \
train_model_tfidf(GaussianNB(), df_simone_train, 100)
model_nb_simone_tfidf = model_nb_simone_tfidf.fit(tfidf_teste(df_train_nb_simone_tfidf, nb_simone_tfidf).toarray(), df_train_nb_simone_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_tfidf(df_simone, nb_simone_tfidf,model_nb_simone_tfidf, 'predict_tfidf_NaiveBayes')


accuracy:
0.6296296296296297
precision:
0.528042328042328
recall:
0.7559957498482088
f1:
0.5046946612164004
matrix_confusion:
[[ 2  0  0]
 [ 7 46  8]
 [12 23 37]]
0.6045905921841417
12
(array([-1,  0,  1]), array([44694, 59470, 26998], dtype=int64))
Wall time: 10.3 s


### Simone - Random Forest - bw

In [49]:
%%time
model_RandomForest_simone_bw, RandomForest_simone_bw, df_train_RandomForest_simone_bw, df_teste_RandomForest_simone_bw = \
train_model_bw(RandomForestClassifier(), df_simone_train, 100)
model_RandomForest_simone_bw = model_RandomForest_simone_bw.fit(Bag_of_words_teste(df_train_RandomForest_simone_bw, RandomForest_simone_bw).toarray(), df_train_RandomForest_simone_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_bw(df_simone, RandomForest_simone_bw,model_RandomForest_simone_bw, 'predict_bw_RandomForest')


accuracy:
0.6444444444444445
precision:
0.5236051228002938
recall:
0.7660898603521554
f1:
0.52517150110733
matrix_confusion:
[[ 2  0  0]
 [ 3 47 11]
 [10 24 38]]
0.6048852712733213
1
(array([-1,  0,  1]), array([ 19807, 100847,  10508], dtype=int64))
Wall time: 29 s


### Simone - Random Forest - tfidf

In [50]:
%%time
model_RandomForest_simone_tfidf, RandomForest_simone_tfidf, df_train_RandomForest_simone_tfidf, df_teste_RandomForest_simone_tfidf = \
train_model_tfidf(RandomForestClassifier(), df_simone_train, 100)
model_RandomForest_simone_tfidf = model_RandomForest_simone_tfidf.fit(tfidf_teste(df_train_RandomForest_simone_tfidf, RandomForest_simone_tfidf).toarray(), df_train_RandomForest_simone_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_tfidf(df_simone, RandomForest_simone_tfidf,model_RandomForest_simone_tfidf, 'predict_tfidf_RandomForest')


accuracy:
0.674074074074074
precision:
0.5407843137254902
recall:
0.7762598664238008
f1:
0.5400167084377611
matrix_confusion:
[[ 2  0  0]
 [ 8 37 16]
 [ 7 13 52]]
0.6327837406652815
7
(array([-1,  0,  1]), array([32810, 49438, 48914], dtype=int64))
Wall time: 29 s


### Simone - SVM - bw

In [51]:
%%time
model_svc_simone_bw, svc_simone_bw, df_train_svc_simone_bw, df_teste_svc_simone_bw = \
train_model_bw(LinearSVC(), df_simone_train, 100)
model_svc_simone_bw = model_svc_simone_bw.fit(Bag_of_words_teste(df_train_svc_simone_bw, svc_simone_bw).toarray(), df_train_svc_simone_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_bw(df_simone, svc_simone_bw,model_svc_simone_bw, 'predict_bw_SVM')

accuracy:
0.6444444444444445
precision:
0.5623572559806619
recall:
0.7560716454159078
f1:
0.5147818599311137
matrix_confusion:
[[ 2  0  0]
 [14 35 12]
 [14  8 50]]
0.6315871703073883
11
(array([-1,  0,  1]), array([40151, 61066, 29945], dtype=int64))
Wall time: 7.03 s


### Simone - SVM - tfidf

In [52]:
%%time
model_svc_simone_tfidf, svc_simone_tfidf, df_train_svc_simone_tfidf, df_teste_svc_simone_tfidf = \
train_model_tfidf(LinearSVC(), df_simone_train, 100)
model_svc_simone_tfidf = model_svc_simone_tfidf.fit(tfidf_teste(df_train_svc_simone_tfidf, svc_simone_tfidf).toarray(), df_train_svc_simone_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_tfidf(df_simone, svc_simone_tfidf,model_svc_simone_tfidf, 'predict_tfidf_SVM')

accuracy:
0.6592592592592592
precision:
0.5532679738562091
recall:
0.7728445658773527
f1:
0.5267686243295999
matrix_confusion:
[[ 2  0  0]
 [ 9 44  8]
 [13 16 43]]
0.6280351058306053
13
(array([-1,  0,  1]), array([41356, 57685, 32121], dtype=int64))
Wall time: 6.97 s


### Simone - MLP - bw

In [53]:
%%time
model_mlp_simone_bw, mlp_simone_bw, df_train_mlp_simone_bw, df_teste_mlp_simone_bw = \
train_model_bw(MLPClassifier(max_iter=400), df_simone_train, 100)
model_mlp_simone_bw = model_mlp_simone_bw.fit(Bag_of_words_teste(df_train_mlp_simone_bw, mlp_simone_bw).toarray(), df_train_mlp_simone_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_bw(df_simone, mlp_simone_bw,model_mlp_simone_bw, 'predict_bw_MLP')

accuracy:
0.6888888888888889
precision:
0.5500605326876514
recall:
0.6318306010928961
f1:
0.5257891414141415
matrix_confusion:
[[ 1  0  1]
 [ 4 47 10]
 [15 12 45]]
0.6674181912608373
7
(array([-1,  0,  1]), array([32141, 68302, 30719], dtype=int64))
Wall time: 1min 39s


### Simone - MLP - tfidf

In [54]:
%%time
model_mlp_simone_tfidf, mlp_simone_tfidf, df_train_mlp_simone_tfidf, df_teste_mlp_simone_tfidf = \
train_model_tfidf(MLPClassifier(max_iter=400), df_simone_train, 100)
model_mlp_simone_tfidf = model_mlp_simone_tfidf.fit(Bag_of_words_teste(df_train_mlp_simone_tfidf, mlp_simone_tfidf).toarray(), df_train_mlp_simone_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_tfidf(df_simone, mlp_simone_tfidf,model_mlp_simone_tfidf, 'predict_tfidf_MLP')

accuracy:
0.6814814814814815
precision:
0.5544677544677544
recall:
0.6213570127504554
f1:
0.520601278302343
matrix_confusion:
[[ 1  0  1]
 [ 8 40 13]
 [13  8 51]]
0.5944768817505086
14
(array([-1,  0,  1]), array([38912, 31518, 60732], dtype=int64))
Wall time: 2min 3s


### Simone - KNN - bw

In [55]:
%%time
model_knn_simone_bw, knn_simone_bw, df_train_knn_simone_bw, df_teste_knn_simone_bw = \
train_model_bw(KNeighborsClassifier(), df_simone_train, 100)
model_knn_simone_bw = model_knn_simone_bw.fit(Bag_of_words_teste(df_train_knn_simone_bw, knn_simone_bw).toarray(), df_train_knn_simone_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_bw(df_simone, knn_simone_bw,model_knn_simone_bw, 'predict_bw_KNN')

accuracy:
0.48148148148148145
precision:
0.5193947525342875
recall:
0.49218275652701876
f1:
0.40932370099036763
matrix_confusion:
[[ 1  0  1]
 [16 35 10]
 [35  8 29]]
0.6629099743877609
2
(array([-1,  0,  1]), array([  7000,   7389, 116773], dtype=int64))
Wall time: 14.4 s


### Simone - KNN - tfidf

In [56]:
%%time
model_knn_simone_tfidf, knn_simone_tfidf, df_train_knn_simone_tfidf, df_teste_knn_simone_tfidf = \
train_model_tfidf(KNeighborsClassifier(), df_simone_train, 100)
model_knn_simone_tfidf = model_knn_simone_tfidf.fit(Bag_of_words_teste(df_train_knn_simone_tfidf, knn_simone_tfidf).toarray(), df_train_knn_simone_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_tfidf(df_simone, knn_simone_tfidf,model_knn_simone_tfidf, 'predict_tfidf_KNN')

accuracy:
0.6148148148148148
precision:
0.5269862807176241
recall:
0.7450667880995749
f1:
0.49309239610963757
matrix_confusion:
[[ 2  0  0]
 [10 44  7]
 [12 23 37]]
0.5949900699354128
12
(array([-1,  0,  1]), array([43923, 64888, 22351], dtype=int64))
Wall time: 30.9 s


# Ciro

### Ciro - Arvore de Decisao - bw

In [57]:
%%time
model_tree_ciro_bw, tree_ciro_bw, df_train_tree_ciro_bw, df_teste_tree_ciro_bw = \
train_model_bw(tree.DecisionTreeClassifier(), df_ciro_train, 100)
model_tree_ciro_bw = model_tree_ciro_bw.fit(Bag_of_words_teste(df_train_tree_ciro_bw, tree_ciro_bw), df_train_tree_ciro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_bw(df_ciro, tree_ciro_bw,model_tree_ciro_bw, 'predict_bw_ArvoreDeDecisao')


accuracy:
0.28656009473060984
precision:
0.33467291271846134
recall:
0.3002530564956249
f1:
0.15164556012169253
matrix_confusion:
[[   0    1    1]
 [   8   16    2]
 [1957 2851 1920]]
0.39563912385063027
1609
(array([-1,  0,  1]), array([126080, 183774, 188212], dtype=int64))
Wall time: 45 s


### Ciro - Arvore de Decisao - tfidf

In [58]:
%%time
model_tree_ciro_tfidf, tree_ciro_tfidf, df_train_tree_ciro_tfidf, df_teste_tree_ciro_tfidf = \
train_model_tfidf(tree.DecisionTreeClassifier(), df_ciro_train, 100)
model_tree_ciro_tfidf = model_tree_ciro_tfidf.fit(tfidf_teste(df_train_tree_ciro_tfidf, tree_ciro_tfidf), df_train_tree_ciro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_tfidf(df_ciro, tree_ciro_tfidf,model_tree_ciro_tfidf, 'predict_tfidf_ArvoreDeDecisao')


accuracy:
0.4698046181172291
precision:
0.33559904548693487
recall:
0.6820520442696424
f1:
0.21888229383324884
matrix_confusion:
[[   2    0    0]
 [   3   15    8]
 [1754 1817 3157]]
0.4265845004267638
1754
(array([-1,  0,  1]), array([205602, 134024, 158440], dtype=int64))
Wall time: 45.4 s


### Ciro - Naive Bayes - bw

In [59]:
%%time
model_nb_ciro_bw, nb_ciro_bw, df_train_nb_ciro_bw, df_teste_nb_ciro_bw = \
train_model_bw(GaussianNB(), df_ciro_train, 100)
model_nb_ciro_bw = model_nb_ciro_bw.fit(Bag_of_words_teste(df_train_nb_ciro_bw, nb_ciro_bw).toarray(), df_train_nb_ciro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_bw(df_ciro, nb_ciro_bw,model_nb_ciro_bw, 'predict_bw_NaiveBayes')


accuracy:
0.4732089994079337
precision:
0.33593514848800465
recall:
0.6193367175828531
f1:
0.220744521256287
matrix_confusion:
[[   2    0    0]
 [   8   10    8]
 [2502 1041 3185]]
0.3876135890079022
2297
(array([-1,  0,  1]), array([190682, 129915, 177469], dtype=int64))
Wall time: 1min 27s


### Ciro - Naive Bayes - tfidf

In [60]:
%%time
model_nb_ciro_tfidf, nb_ciro_tfidf, df_train_nb_ciro_tfidf, df_teste_nb_ciro_tfidf = \
train_model_tfidf(GaussianNB(), df_ciro_train, 100)
model_nb_ciro_tfidf = model_nb_ciro_tfidf.fit(tfidf_teste(df_train_nb_ciro_tfidf, nb_ciro_tfidf).toarray(), df_train_nb_ciro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_tfidf(df_ciro, nb_ciro_tfidf,model_nb_ciro_tfidf, 'predict_tfidf_NaiveBayes')


accuracy:
0.49985198342214326
precision:
0.336964314578899
recall:
0.7048804841611025
f1:
0.23000654748328112
matrix_confusion:
[[   2    0    0]
 [   5   16    5]
 [1978 1391 3359]]
0.44292583241135647
1978
(array([-1,  0,  1]), array([157390, 150551, 190125], dtype=int64))
Wall time: 1min 31s


### Ciro - Random Forest - bw

In [61]:
%%time
model_RandomForest_ciro_bw, RandomForest_ciro_bw, df_train_RandomForest_ciro_bw, df_teste_RandomForest_ciro_bw = \
train_model_bw(RandomForestClassifier(), df_ciro_train, 100)
model_RandomForest_ciro_bw = model_RandomForest_ciro_bw.fit(Bag_of_words_teste(df_train_RandomForest_ciro_bw, RandomForest_ciro_bw).toarray(), df_train_RandomForest_ciro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_bw(df_ciro, RandomForest_ciro_bw,model_RandomForest_ciro_bw, 'predict_bw_RandomForest')


accuracy:
0.3845470692717584
precision:
0.3366132132635921
recall:
0.6790565261136009
f1:
0.19190537170200664
matrix_confusion:
[[   2    0    0]
 [   6   17    3]
 [2500 1649 2579]]
0.45749387834973876
1573
(array([-1,  0,  1]), array([123245, 149778, 225043], dtype=int64))
Wall time: 2min 3s


### Ciro - Random Forest - tfidf

In [62]:
%%time
model_RandomForest_ciro_tfidf, RandomForest_ciro_tfidf, df_train_RandomForest_ciro_tfidf, df_teste_RandomForest_ciro_tfidf = \
train_model_tfidf(RandomForestClassifier(), df_ciro_train, 100)
model_RandomForest_ciro_tfidf = model_RandomForest_ciro_tfidf.fit(tfidf_teste(df_train_RandomForest_ciro_tfidf, RandomForest_ciro_tfidf).toarray(), df_train_RandomForest_ciro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_tfidf(df_ciro, RandomForest_ciro_tfidf,model_RandomForest_ciro_tfidf, 'predict_tfidf_RandomForest')


accuracy:
0.38972764949674366
precision:
0.3363541886896759
recall:
0.6935615415104119
f1:
0.1931532597422734
matrix_confusion:
[[   2    0    0]
 [   5   18    3]
 [2198 1917 2613]]
0.4031991598597762
2198
(array([-1,  0,  1]), array([197877, 194728, 105461], dtype=int64))
Wall time: 1min 56s


### Ciro - SVM - bw

In [63]:
%%time
model_svc_ciro_bw, svc_ciro_bw, df_train_svc_ciro_bw, df_teste_svc_ciro_bw = \
train_model_bw(LinearSVC(), df_ciro_train, 100)
model_svc_ciro_bw = model_svc_ciro_bw.fit(Bag_of_words_teste(df_train_svc_ciro_bw, svc_ciro_bw).toarray(), df_train_svc_ciro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_bw(df_ciro, svc_ciro_bw,model_svc_ciro_bw, 'predict_bw_SVM')

accuracy:
0.422291296625222
precision:
0.3369752829907065
recall:
0.6789193268087441
f1:
0.20522440411164308
matrix_confusion:
[[   2    0    0]
 [   8   16    2]
 [2429 1464 2835]]
0.43872051832945386
1814
(array([-1,  0,  1]), array([146807, 180671, 170588], dtype=int64))
Wall time: 48.4 s


### Ciro - SVM - tfidf

In [64]:
%%time
model_svc_ciro_tfidf, svc_ciro_tfidf, df_train_svc_ciro_tfidf, df_teste_svc_ciro_tfidf = \
train_model_tfidf(LinearSVC(), df_ciro_train, 100)
model_svc_ciro_tfidf = model_svc_ciro_tfidf.fit(tfidf_teste(df_train_svc_ciro_tfidf, svc_ciro_tfidf).toarray(), df_train_svc_ciro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_tfidf(df_ciro, svc_ciro_tfidf,model_svc_ciro_tfidf, 'predict_tfidf_SVM')

accuracy:
0.5047365304914151
precision:
0.3363388176068713
recall:
0.5398983200707339
f1:
0.23043927412910492
matrix_confusion:
[[   1    1    0]
 [   4   16    6]
 [1787 1548 3393]]
0.40285323557453134
1787
(array([-1,  0,  1]), array([139285, 183303, 175478], dtype=int64))
Wall time: 46.4 s


### Ciro - MLP - bw

In [65]:
%%time
model_mlp_ciro_bw, mlp_ciro_bw, df_train_mlp_ciro_bw, df_teste_mlp_ciro_bw = \
train_model_bw(MLPClassifier(max_iter=400), df_ciro_train, 100)
model_mlp_ciro_bw = model_mlp_ciro_bw.fit(Bag_of_words_teste(df_train_mlp_ciro_bw, mlp_ciro_bw).toarray(), df_train_mlp_ciro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_bw(df_ciro, mlp_ciro_bw,model_mlp_ciro_bw, 'predict_bw_MLP')

accuracy:
0.4635879218472469
precision:
0.3363654523608996
recall:
0.5261250342998262
f1:
0.21844585495660276
matrix_confusion:
[[   1    0    1]
 [   3   16    7]
 [2201 1412 3115]]
0.4455410179104286
1775
(array([-1,  0,  1]), array([152025, 142870, 203171], dtype=int64))
Wall time: 4min 39s


### Ciro - MLP - tfidf

In [66]:
%%time
model_mlp_ciro_tfidf, mlp_ciro_tfidf, df_train_mlp_ciro_tfidf, df_teste_mlp_ciro_tfidf = \
train_model_tfidf(MLPClassifier(max_iter=400), df_ciro_train, 100)
model_mlp_ciro_tfidf = model_mlp_ciro_tfidf.fit(Bag_of_words_teste(df_train_mlp_ciro_tfidf, mlp_ciro_tfidf).toarray(), df_train_mlp_ciro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_tfidf(df_ciro, mlp_ciro_tfidf,model_mlp_ciro_tfidf, 'predict_tfidf_MLP')

accuracy:
0.5201302545885139
precision:
0.335820993298076
recall:
0.6605841641513461
f1:
0.23434192749187632
matrix_confusion:
[[   2    0    0]
 [   6   12    8]
 [1856 1372 3500]]
0.43771933488245307
1856
(array([-1,  0,  1]), array([156087, 162440, 179539], dtype=int64))
Wall time: 5min 6s


### Ciro - KNN - bw

In [67]:
%%time
model_knn_ciro_bw, knn_ciro_bw, df_train_knn_ciro_bw, df_teste_knn_ciro_bw = \
train_model_bw(KNeighborsClassifier(), df_ciro_train, 100)
model_knn_ciro_bw = model_knn_ciro_bw.fit(Bag_of_words_teste(df_train_knn_ciro_bw, knn_ciro_bw).toarray(), df_train_knn_ciro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_bw(df_ciro, knn_ciro_bw,model_knn_ciro_bw, 'predict_bw_KNN')

accuracy:
0.012137359384251036
precision:
0.3350121246036187
recall:
0.23394005914814475
f1:
0.00961523360816647
matrix_confusion:
[[   0    2    0]
 [   8   18    0]
 [3110 3554   64]]
0.4097075980234055
1286
(array([-1,  0,  1]), array([ 95788,  54268, 348010], dtype=int64))
Wall time: 1min 48s


### Ciro - KNN - tfidf

In [68]:
%%time
model_knn_ciro_tfidf, knn_ciro_tfidf, df_train_knn_ciro_tfidf, df_teste_knn_ciro_tfidf = \
train_model_tfidf(KNeighborsClassifier(), df_ciro_train, 100)
model_knn_ciro_tfidf = model_knn_ciro_tfidf.fit(Bag_of_words_teste(df_train_knn_ciro_tfidf, knn_ciro_tfidf).toarray(), df_train_knn_ciro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_tfidf(df_ciro, knn_ciro_tfidf,model_knn_ciro_tfidf, 'predict_tfidf_KNN')

accuracy:
0.39742451154529307
precision:
0.33507961058417446
recall:
0.6067410591786335
f1:
0.1942997178080413
matrix_confusion:
[[   2    0    0]
 [   9   11    6]
 [2414 1642 2672]]
0.38338622477903556
2414
(array([-1,  0,  1]), array([199838, 180045, 118183], dtype=int64))
Wall time: 2min 3s


In [69]:
df_bolsonaro.to_excel('bolsonaro_predict3.xlsx')
df_lula.to_excel('lula_predict3.xlsx')
df_simone.to_excel('simone_predict3.xlsx')
df_ciro.to_excel('ciro_predict3.xlsx')

In [70]:
def VotacaoPredict(df):
    for i in df.index:
        pos=0
        neu=0
        neg=0
        for j in ['predict_tfidf_MLP','predict_bw_MLP',
                  'predict_tfidf_SVM','predict_bw_SVM',
                 'predict_tfidf_RandomForest','predict_bw_RandomForest',
                 'predict_tfidf_NaiveBayes','predict_bw_NaiveBayes',
                 'predict_tfidf_ArvoreDeDecisao','predict_bw_ArvoreDeDecisao',
                 'predict_tfidf_KNN', 'predict_bw_KNN']:
            if (df.loc[i,j]==1):
                pos+=1
            elif (df.loc[i,j]==0):
                neu+=1    
            else:
                neg+=1
        
        if((neu>=neg)& (neu>=pos)):
            df.loc[i,'VotacaoPredict_norm'] = 0
        elif((neg>=pos)& (neg>=neu)):
            df.loc[i,'VotacaoPredict_norm'] = -1
        else:
            df.loc[i,'VotacaoPredict_norm'] = 1     
    #print(df['VotacaoPredict_ajust'].value_counts())
    print(df['VotacaoPredict_norm'].value_counts())        
    return df

In [71]:
df_bolsonaro_predict2 = VotacaoPredict(df_bolsonaro)
df_lula_predict2 = VotacaoPredict(df_lula)
df_simone_predict2 = VotacaoPredict(df_simone)
df_ciro_predict2 = VotacaoPredict(df_ciro)

 1.0    546907
 0.0    302082
-1.0     97449
Name: VotacaoPredict_norm, dtype: int64
 0.0    540879
 1.0    284524
-1.0    106327
Name: VotacaoPredict_norm, dtype: int64
 0.0    61606
-1.0    35215
 1.0    34341
Name: VotacaoPredict_norm, dtype: int64
 0.0    175705
-1.0    161910
 1.0    160451
Name: VotacaoPredict_norm, dtype: int64


In [74]:
df_bolsonaro_predict2.to_excel('bolsonaro_resultado_equilibrio2.xlsx')
df_lula_predict2.to_excel('lula_resultado_equilibrio2.xlsx')
df_simone_predict2.to_excel('simone_resultado_equilibrio2.xlsx')
df_ciro_predict2.to_excel('ciro_resultado_equilibrio2.xlsx')

In [ ]:
# df_bolsonaro = pd.read_excel('bolsonaro_predict1.xlsx', index_col=0)
# df_lula = pd.read_excel('lula_predict1.xlsx', index_col=0)
# df_simone = pd.read_excel('simone_predict1.xlsx', index_col=0)
# df_ciro = pd.read_excel('ciro_predict1.xlsx', index_col=0)